In [5]:
import numpy as np

W = np.random.randn(5, 3)  # example matrix
U, S, Vt = np.linalg.svd(W, full_matrices=False)

u = U[:, 0]      # Left singular vector
v = Vt[0, :]     # Right singular vector (note: Vt = V.T)
print(u)
print(v)
print(S[0])  # Singular value

[-0.4367726  -0.03793902 -0.31824461  0.6984112  -0.46768845]
[ 0.04641084 -0.2999522   0.9528246 ]
3.3651089804275913


In [6]:
def top_singular_vectors(W, num_iter=100):
    v = np.random.randn(W.shape[1])
    v /= np.linalg.norm(v)
    
    for _ in range(num_iter):
        u = W @ v
        u /= np.linalg.norm(u)
        v = W.T @ u
        v /= np.linalg.norm(v)
    
    return u, v

u, v = top_singular_vectors(W)
print(u)
print(v)
print(u.T @ W @ v)


[ 0.4367726   0.03793902  0.31824461 -0.6984112   0.46768845]
[-0.04641084  0.2999522  -0.9528246 ]
3.3651089804275918


In [64]:
import numpy as np
def u1s1v1t(W, num_iter=30):
    """Power iteration using PyTorch operations"""
    v = np.random.randn(W.shape[1])
    v /= np.linalg.norm(v)
    
    for _ in range(num_iter):
        u = W @ v
        u /= np.linalg.norm(u)
        v = W.T @ u
        v /= np.linalg.norm(v)
    sigma1 = (u.T @ W @ v)
    u = u.reshape(-1, 1)         # shape: (5, 1)
    v = v.reshape(-1, 1)         # shape: (3, 1)
    return sigma1 * u @ v.T

In [ ]:
import cupy as cp
from cupyx.scipy.sparse.linalg import svds as cupyx_svds

def u1s1v1t_svds(W, num_iter=30):
    assert (W.shape[0] > 3 and W.shape[1] > 3), "dimentions of W must be grater than 3"
    U, S, Vt = cupyx_svds(W,k=1,maxiter=num_iter,which='LM')

    sigma1 = S[0]  # Largest singular value
    u = U[:, 0].reshape(-1, 1)  # Reshape to column vector
    v = Vt[0, :].reshape(-1, 1) # Reshape to column vector
    
    return sigma1 * (u @ v.T)

In [ ]:
# import numpy as np
# from petsc4py import PETSc
# from slepc4py import SLEPc

# def u1s1v1t_slepc(W_numpy, num_iter=30):
#     """Compute rank-1 approximation using SLEPc's Lanczos SVD"""
#     m, n = W_numpy.shape
    
#     # Convert to PETSc matrix (sparse-friendly)
#     A = PETSc.Mat().createDense((m, n), array=W_numpy)
#     A.assemble()

#     # Create and configure SVD solver
#     svd = SLEPc.SVD().create()
#     svd.setOperator(A)
#     svd.setType(SLEPc.SVD.Type.LANCZOS)  # Directly use Lanczos SVD method
#     svd.setDimensions(1)                 # Request 1 singular value
#     svd.setTolerances(max_it=num_iter)    # Set iteration limit
#     svd.setWhichSingularTriplets(SLEPc.SVD.Which.LARGEST)
    
#     # Solve SVD
#     svd.solve()

#     # Check convergence
#     if svd.getConverged() < 1:
#         raise RuntimeError("SVD failed to converge")

#     # Extract results
#     sigma = svd.getSingularValue(0)
#     u, v = A.getVecLeft(), A.getVecRight()
#     svd.getSingularTriplet(0, u, v)

#     return sigma * np.outer(u.getArray(), v.getArray())

In [ ]:
# test_matr = cp.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
import time
np.random.seed(42)
W_numpy = np.random.randn(1000, 1000)
W_gpu = cp.asarray(W_numpy)

start = time.time()
result_gpu = u1s1v1t_svds(W_gpu, num_iter=3)
time_svds = time.time() - start

start = time.time()
result_cpu = u1s1v1t(W_numpy, num_iter=1000)
time_power_iter = time.time() - start

# result_slepc = u1s1v1t_slepc(W_numpy, num_iter=3)

U, S, Vt = np.linalg.svd(W_numpy)
result_precise = S[0] * (U[:, 0].reshape(-1, 1) @ Vt[0, :].reshape(-1, 1).T)

np.set_printoptions(formatter={'float_kind': '{:.2f}'.format})
# print("svds_gpu:\n",result_gpu, "\n\n", "power_iters_cpu:\n",result_cpu, "\n\n", "precise:\n",result_precise)
print("relative error of power iterations:\n", np.linalg.norm(result_cpu-result_precise)/np.linalg.norm(result_precise))
print("relative error of svds:\n", np.linalg.norm(cp.asnumpy(result_gpu)-result_precise)/np.linalg.norm(result_precise))
# print("relative error of slepc:\n",            np.linalg.norm(result_slepc-result_precise)/np.linalg.norm(result_precise))
print(f'time for svds: {time_svds}')
print(f'time for power iterations: {time_power_iter}')


In [65]:
u1s1v1t(np.array([[1, 2, 3], [4, 5, 6]]), num_iter=30)

array([[1.57454629, 2.08011388, 2.58568148],
       [3.75936076, 4.96644562, 6.17353048]])